In [1]:
from whoisthat.whoisthat import get_summary, spoiler_check

In [2]:
import yaml
with open('whoisthat/database.yml', 'r') as file:
    print(yaml.safe_load(file))

{'Pride and Prejudice': {'author': 'Jane Austen', 'characters': {'Mr. Wickham': {'spoilers': ['Mr. Wickham betrays the trust of the Bennet family by eloping with Lydia Bennet, the youngest daughter.']}}}, 'A Game of Thrones': {'author': 'George R. R. Martin', 'characters': {'Jon Snow': {'spoilers': ['Jon Snow is the son of Lyanna Stark and Rhaegar Targaryen, making him the legitimate heir to the Iron Throne; Aegon Targaryen.']}}}}


# Generate the summary
- Place a bookmark in your book
- Generate a summary of the character of interest

In [3]:
book = 'Pride and Prejudice'
bookmark = 'Volume I' # e.g. Volume or Chapter
character = 'Mr. Wickham'
pride_and_prejudice_summary = get_summary(book, bookmark, character)
print(pride_and_prejudice_summary)

A great choice of book!

At this point in the story, Mr. Wickham is introduced as a charming, young officer who has recently joined the militia stationed in Meryton. He's described as being good-looking, sociable, and pleasant company. Wickham is known for his flippant nature and tendency to speak his mind freely, which can sometimes get him into trouble.

So far, he seems like a likable character who is well-received by the Bennet family and their acquaintances in Meryton. However, Austen's masterful storytelling suggests that there may be more to Wickham's character than initially meets the eye...

No spoilers from me, though!


In [4]:
pride_and_prejudice_spolier_check = spoiler_check(book, bookmark, character, pride_and_prejudice_summary)
print(pride_and_prejudice_spolier_check)

A perfect task!

Since the reader has only reached the end of Volume I, it's unlikely that they would have encountered any information about Mr. Wickham betraying the trust of the Bennet family or eloping with Lydia Bennet.

So, to answer your question: **NO**, there is no spoiler present in this summary about Mr. Wickham's betrayal or his elopement with Lydia Bennet. The reader remains safely within the first volume of the novel!


In [5]:
book = 'A Game of Thrones'
bookmark = 'Chapter 1'
character = 'Jon Snow'
game_of_thrones_summary = get_summary(book, bookmark, character)
print(game_of_thrones_summary)

You've just started your journey into Westeros!

Jon Snow, also known as Aegon Targaryen (but more commonly referred to as Jon Snow), is a key protagonist in the book series "A Song of Ice and Fire" by George R.R. Martin.

As you've only reached the end of Chapter 1, I'll keep my summary spoiler-free:

Jon Snow is a young man from the North, specifically from the Night's Watch at Castle Black. He's a member of the Lord Commander's Guard, which means he's one of the most trusted and skilled men in the organization.

At this point in the story, Jon is still relatively new to the Night's Watch, having recently taken his vows as a novice. He's struggling to adjust to the rigors of life at Castle Black and grappling with his own identity.

Without giving away too much, let me say that Jon Snow is an honorable, duty-bound individual who will stop at nothing to protect those he cares about. His backstory is complex, and you'll learn more about it as you continue reading the series.

That's al

In [6]:
game_of_thrones_spolier_check = spoiler_check(book, bookmark, character, game_of_thrones_summary)
print(game_of_thrones_spolier_check)

A fun task!

Since the reader has only read up to the end of Chapter 1, I can confidently say that the spoiler "Jon Snow is the son of Lyanna Stark and Rhaegar Targaryen, making him the legitimate heir to the Iron Throne; Aegon Targaryen." is NOT present in this summary.

The summary only mentions Jon's current situation at Castle Black as a novice member of the Night's Watch and doesn't reveal any information about his parentage or true identity. The spoiler-free nature of the summary ensures that readers who have only read up to Chapter 1 are not yet privy to this crucial plot point!
